In [1]:
import pandas as pd 
import os
import traceback
import json

In [2]:
print(pd.__version__)

2.3.2


In [3]:
import sys
print(sys.executable)

d:\GenAI\MCQ Generator\env\python.exe


In [4]:
!{sys.executable} -m pip install langchain

'd:\GenAI\MCQ' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from langchain_openai import ChatOpenAI

In [7]:
key = os.getenv("OPENAI_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1" 

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
client = ChatOpenAI(
    model = "deepseek/deepseek-chat-v3.1:free",
    temperature = 0.8
)

response = client.invoke("Who is the Prime Minister of India ?")
print(response.content)

Of course! The Prime Minister of India is **Narendra Modi**.

He is the head of the government and the leader of the executive branch.

Here are some key details:

*   **Political Party:** Bharatiya Janata Party (BJP)
*   **Took Office:** 26th May 2014
*   **Current Term:** This is his second consecutive term, which began on **30th May 2019**.

He was previously the Chief Minister of the state of Gujarat from 2001 to 2014.


In [10]:
response2 = client.invoke("what is the color of sunlight?")
print(response2.content)

That's an excellent question! The answer is more fascinating than you might think, and it depends on how you look at it.

The short answer is: **To human eyes on Earth, the Sun's light appears white.**

However, the full story involves physics, biology, and perspective. Here’s a breakdown:

### 1. The Scientific View: It's All the Colors
Sunlight is the full spectrum of visible light. When sunlight passes through a prism, it separates into all the colors of the rainbow: **red, orange, yellow, green, blue, indigo, and violet**. This means sunlight contains all of these colors combined.

The Sun emits energy across a wide range of wavelengths, but the peak of its emission is in the green portion of the spectrum.

### 2. The Biological View: Why We See It as White
Our eyes have three types of color receptors (cones) that are most sensitive to red, green, and blue light. When all these receptors are stimulated equally by a light source containing all wavelengths, our brain interprets that 

In [11]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
import PyPDF2
import re

In [12]:
response_json = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer here",
    },

    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer here",
    },

    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer here",
    },

}

In [13]:
quiz_template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [14]:
from langchain.prompts import PromptTemplate
quiz_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=quiz_template
)


In [15]:
quiz_chain = LLMChain(llm=client, prompt=quiz_prompt, output_key="quiz", verbose=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_17688\2053275858.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=client, prompt=quiz_prompt, output_key="quiz", verbose=True)


In [16]:
evaluate_template="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:"""

In [17]:
evaluate_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = evaluate_template
)

In [18]:
eval_chain = LLMChain(llm=client, prompt=evaluate_prompt, output_key="review", verbose=True)

In [19]:
final_mcq_generator = SequentialChain(chains=[quiz_chain, eval_chain],
                                      input_variables = ["text", "number", "subject", "tone", "response_json"],
                                      output_variables = ["quiz", "review"], verbose = True)

In [20]:
file_path = r"D:\GenAI\MCQ Generator\Roman_history.txt"

In [21]:
with open(file_path,'r', encoding='utf-8') as file:
    Text = file.read()
print(Text)

In modern historiography, ancient Rome is the Roman civilisation from the founding of the Italian city of Rome in the 8th century BC to the collapse of the Western Roman Empire in the 5th century AD. It encompasses the Roman Kingdom (753–509 BC), the Roman Republic (509‍–‍27 BC), and the Roman Empire (27 BC – 476 AD) until the fall of the western empire.[1][a]

Ancient Rome began as an Italic settlement, traditionally dated to 753 BC, beside the River Tiber in the Italian peninsula. The settlement grew into the city and polity of Rome, and came to control its neighbours through a combination of treaties and military strength. It eventually controlled the Italian Peninsula, assimilating the Greek culture of southern Italy (Magna Graecia) and the Etruscan culture, and then became the dominant power in the Mediterranean region and parts of Europe. At its height it controlled the North African coast, Egypt, Southern Europe, and most of Western Europe, the Balkans, Crimea, and much of the M

In [22]:
num = 5
sub = "Ancinet Rome"
tone = "Medium"

In [23]:
with get_openai_callback() as cb:
    response = final_mcq_generator(
        {
        "text":Text,
        "number":num,
        "subject":sub,
        "tone":tone,
        "response_json":json.dumps(response_json)
        }
    )


C:\Users\Dell\AppData\Local\Temp\ipykernel_17688\4106291851.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = final_mcq_generator(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:In modern historiography, ancient Rome is the Roman civilisation from the founding of the Italian city of Rome in the 8th century BC to the collapse of the Western Roman Empire in the 5th century AD. It encompasses the Roman Kingdom (753–509 BC), the Roman Republic (509‍–‍27 BC), and the Roman Empire (27 BC – 476 AD) until the fall of the western empire.[1][a]

Ancient Rome began as an Italic settlement, traditionally dated to 753 BC, beside the River Tiber in the Italian peninsula. The settlement grew into the city and polity of Rome, and came to control its neighbours through a combination of treaties and military strength. It eventually controlled the Italian Peninsula, assimilating the Greek culture of southern Italy (Magna Graecia) and the Etruscan culture, and then became the dominant power in the Mediterranean region and parts of Europe. At its height it controlled the Nor

In [24]:
print(cb.total_tokens)
print(cb.total_cost)

2278
0.0


In [25]:
response

{'text': "In modern historiography, ancient Rome is the Roman civilisation from the founding of the Italian city of Rome in the 8th century BC to the collapse of the Western Roman Empire in the 5th century AD. It encompasses the Roman Kingdom (753–509 BC), the Roman Republic (509\u200d–\u200d27 BC), and the Roman Empire (27 BC – 476 AD) until the fall of the western empire.[1][a]\n\nAncient Rome began as an Italic settlement, traditionally dated to 753 BC, beside the River Tiber in the Italian peninsula. The settlement grew into the city and polity of Rome, and came to control its neighbours through a combination of treaties and military strength. It eventually controlled the Italian Peninsula, assimilating the Greek culture of southern Italy (Magna Graecia) and the Etruscan culture, and then became the dominant power in the Mediterranean region and parts of Europe. At its height it controlled the North African coast, Egypt, Southern Europe, and most of Western Europe, the Balkans, Cri

In [26]:
type(response)

dict

In [27]:

quiz_str = response.get("quiz")
print("TYPE:", type(quiz_str))
print("RAW:", repr(quiz_str))

TYPE: <class 'str'>
RAW: '```json\n{\n  "1": {\n    "mcq": "According to modern historiography, what is the correct chronological order of the major periods of Ancient Rome?",\n    "options": {\n      "a": "Roman Kingdom, Roman Republic, Roman Empire",\n      "b": "Roman Republic, Roman Kingdom, Roman Empire",\n      "c": "Roman Empire, Roman Republic, Roman Kingdom",\n      "d": "Roman Kingdom, Roman Empire, Roman Republic"\n    },\n    "correct": "a"\n  },\n  "2": {\n    "mcq": "What was the estimated population of the Roman Empire at its peak in AD 117?",\n    "options": {\n      "a": "20 to 40 million inhabitants",\n      "b": "50 to 90 million inhabitants",\n      "c": "10 to 20 million inhabitants",\n      "d": "100 to 120 million inhabitants"\n    },\n    "correct": "b"\n  },\n  "3": {\n    "mcq": "According to Roman legend, who were the twin founders of Rome, said to have been suckled by a she-wolf?",\n    "options": {\n      "a": "Aeneas and Romulus",\n      "b": "Remus and Ae

In [28]:
cleaned = re.sub(r"^```(?:json)?\n?", "", quiz_str)   # remove starting ```json
cleaned = re.sub(r"\n?```$", "", cleaned.strip())     # remove ending ```
quiz = json.loads(cleaned)
type(quiz)

dict

In [29]:
print(quiz)

{'1': {'mcq': 'According to modern historiography, what is the correct chronological order of the major periods of Ancient Rome?', 'options': {'a': 'Roman Kingdom, Roman Republic, Roman Empire', 'b': 'Roman Republic, Roman Kingdom, Roman Empire', 'c': 'Roman Empire, Roman Republic, Roman Kingdom', 'd': 'Roman Kingdom, Roman Empire, Roman Republic'}, 'correct': 'a'}, '2': {'mcq': 'What was the estimated population of the Roman Empire at its peak in AD 117?', 'options': {'a': '20 to 40 million inhabitants', 'b': '50 to 90 million inhabitants', 'c': '10 to 20 million inhabitants', 'd': '100 to 120 million inhabitants'}, 'correct': 'b'}, '3': {'mcq': 'According to Roman legend, who were the twin founders of Rome, said to have been suckled by a she-wolf?', 'options': {'a': 'Aeneas and Romulus', 'b': 'Remus and Aeneas', 'c': 'Romulus and Remus', 'd': 'Mars and Jupiter'}, 'correct': 'c'}, '4': {'mcq': 'What was the name of the system of government created by the Romans, which is an inspiratio

In [30]:
quiz_table = []
for key, value in quiz.items():
    mcq = value["mcq"]
    option = "  ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
        )
    correct = value["correct"]
    quiz_table.append({"MCQ":mcq, "Choice":option, "Correct":correct})

In [31]:
quiz_table

[{'MCQ': 'According to modern historiography, what is the correct chronological order of the major periods of Ancient Rome?',
  'Choice': 'a : Roman Kingdom, Roman Republic, Roman Empire  b : Roman Republic, Roman Kingdom, Roman Empire  c : Roman Empire, Roman Republic, Roman Kingdom  d : Roman Kingdom, Roman Empire, Roman Republic',
  'Correct': 'a'},
 {'MCQ': 'What was the estimated population of the Roman Empire at its peak in AD 117?',
  'Choice': 'a : 20 to 40 million inhabitants  b : 50 to 90 million inhabitants  c : 10 to 20 million inhabitants  d : 100 to 120 million inhabitants',
  'Correct': 'b'},
 {'MCQ': 'According to Roman legend, who were the twin founders of Rome, said to have been suckled by a she-wolf?',
  'Choice': 'a : Aeneas and Romulus  b : Remus and Aeneas  c : Romulus and Remus  d : Mars and Jupiter',
  'Correct': 'c'},
 {'MCQ': 'What was the name of the system of government created by the Romans, which is an inspiration for modern republics?',
  'Choice': 'a : S

In [32]:
quiz_df = pd.DataFrame(quiz_table)

In [33]:
quiz_df

,MCQ,Choice,Correct
0,"According to modern historiography, what is th...","a : Roman Kingdom, Roman Republic, Roman Empir...",a
1,What was the estimated population of the Roman...,a : 20 to 40 million inhabitants b : 50 to 90...,b
2,"According to Roman legend, who were the twin f...",a : Aeneas and Romulus b : Remus and Aeneas ...,c
3,What was the name of the system of government ...,a : Senatus Populusque Romanus b : Res Public...,b
4,Archaeological evidence suggests that large-sc...,a : 650 BC b : 1000 BC c : 753 BC d : 800 BC,d


In [34]:
quiz_df.to_csv("quiz.csv", index=False)